# You can copy this entire cell

In [6]:
from sklearn import preprocessing
import pandas as pd
import numpy as np

# Loading Dataset
song_data = pd.read_csv("dataset.csv", sep = ',')

# Trimming Dataset
song_data = song_data[song_data['Genre'] != 'n-a']
song_data = song_data[song_data.iloc[:, 0].str.contains("Global") == True].reset_index(drop=True)

# One-Hot Encoding for Artist and Title
enc = preprocessing.OneHotEncoder()
X_PreEnc = song_data[['Title', 'Artist']]
enc.fit(X_PreEnc)
X_Enc = enc.transform(X_PreEnc).toarray()
X_DF = pd.DataFrame(X_Enc, columns=enc.get_feature_names_out(X_PreEnc.columns))

# Filtering One-Hot Encoding for Artist Only
X_DF2 = X_DF.filter(regex='Artist_').reset_index(drop=True)

# Chosen Features (Can change based on feature selection experiments)
X4 = song_data[['Explicit', 'danceability', 'energy', 'loudness', 'speechiness', 'instrumentalness', 'liveliness', 'valence', 'tempo', 'duration_ms', 'acoustics', 'Popu_max']]

# Standard Scaler Preprocessing on chosen features
xcols = ['Explicit', 'danceability', 'energy', 'loudness', 'speechiness', 'instrumentalness', 'liveliness', 'valence', 'tempo', 'duration_ms', 'acoustics', 'Popu_max']
normalized_X = X4.values
std_scalor = preprocessing.StandardScaler()
X_scaled = std_scalor.fit_transform(normalized_X)
normalized_X = pd.DataFrame(X_scaled, columns=xcols)

# Combining both DataFrames (One-Hot Encoding Artists + Preprocessed numerical features)
XT = pd.concat([normalized_X.reset_index(drop=True), X_DF2.reset_index(drop=True)], axis=1)

# Target Labels
y = song_data[['Genre_new']].reset_index(drop=True)

# Combining some genres into larger groups (Can add more here if necessary)
# replace_dict = {
#     'rap':'hip hop',
#     'metal':'rock',
#     'trap':'hip hop',
#     'house': 'dance/electronic'
# }

# ybetter = y['Genre_new'].replace(replace_dict)

# YDF = pd.DataFrame(ybetter)
YDF = pd.DataFrame(y)

# All Done with the Dataset
# The X values are in XT
# The y values are in YDF

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (7,8,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
song_data.isnull().values.any()

True

In [8]:
YDF['Genre_new'].value_counts()

pop                 1706
hip hop             1427
rap                  703
latin                348
rock                 199
r&b/soul             180
dance/electronic     148
house                122
k-pop                119
boy band              86
metal                 81
else                  68
indie                 66
trap                  57
funk                  55
country               28
reggaeton             28
bolero                 2
Name: Genre_new, dtype: int64

## Linear SVM Test

In [413]:
X_train, X_test, y_train, y_test = train_test_split(XT, YDF, test_size=0.2)
clf = LinearSVC(max_iter = 10000, C=1)
#clf = svm.SVC(C=0.1, kernel='linear')
#clf = svm.SVC(C=1, kernel="poly", degree=3)
clf.fit(X_train, y_train.values.ravel())
print("Training Error: " + str(1.0-clf.score(X_train, y_train)))
print("Test Error: " + str(1.0-clf.score(X_test, y_test)))

Training Error: 0.0
Test Error: 0.17603686635944704


Training Error: 0.0

Test Error: 0.18156682027649773

## Logistic Regression Test

In [414]:
clf = LogisticRegression(C=1, solver="lbfgs", penalty="l2", max_iter = 100000000)
clf.fit(X_train,y_train.values.ravel())
#train_accuracy = cross_val_score(clf, X_train2, y_train2).mean()
#test_accuracy = clf.score(X_test2, y_test2)

print("Training Error: " + str(1.0-clf.score(X_train, y_train)))
print("Test Error: " + str(1.0-clf.score(X_test, y_test)))

Training Error: 0.12955278930382663
Test Error: 0.22857142857142854


Training Error: 0.12955278930382663

Test Error: 0.22857142857142854

### 

# Very Messy Experimentation Below

In [13]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

song_data = pd.read_csv("dataset.csv", sep = ',')
song_data = song_data[song_data['Genre'] != 'n-a']
song_data = song_data[song_data.iloc[:, 0].str.contains("Global") == True]
le = preprocessing.LabelEncoder()
X1 = song_data[['Title', 'Artist', 'Explicit', 'danceability', 'energy', 'loudness', 'liveliness', 'valence', 'tempo', 'duration_ms', 'acoustics']]
X = X1
#X = song_data[['Artist', 'Explicit', 'danceability', 'energy', 'loudness', 'liveliness', 'valence', 'tempo', 'duration_ms', 'acoustics']]
X = X.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')
#y = song_data[['Genre']]
y = song_data[['Genre_new']]
y = y.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
#X.head()
X.shape

(5423, 11)

In [155]:
pd.reset_option('max_columns')

In [14]:
X1

,Title,Artist,Explicit,danceability,energy,loudness,liveliness,valence,tempo,duration_ms,acoustics
0,adan y eva,Paulo Londra,False,0.767,0.709,-4.47,0.0676,0.72,171.993,258639,0.323
16,it wont kill ya,The Chainsmokers - Louane,False,0.572,0.53,-8.521,0.127,0.12,170.138,217613,0.0647
39,hymn,Kesha,True,0.488,0.538,-4.974,0.305,0.38,67.037,205600,0.147
58,arrows,Foo Fighters,False,0.515,0.917,-7.312,0.145,0.409,121.988,266187,0.000209
80,talking to myself,Linkin Park,False,0.593,0.712,-6.325,0.128,0.473,124.013,231307,0.00184
...,...,...,...,...,...,...,...,...,...,...,...
83794,lush life,Zara Larsson,False,0.694,0.712,-3.923,0.211,0.799,98.022,201122,0.133
83826,the business,Tiësto,False,0.798,0.62,-7.079,0.112,0.235,120.031,164000,0.414
83839,scooby doo pa pa,Dj Kass,False,0.91,0.754,-2.573,0.122,0.63,120.939,145972,0.0396
83845,positions,Ariana Grande,True,0.737,0.802,-4.771,0.0931,0.682,144.015,172325,0.468


In [371]:
enc = preprocessing.OneHotEncoder()

In [213]:
#X2 = X1['Artist'].values.reshape(-1,1)

In [372]:
X2 = X1[['Title','Artist']]

In [373]:
X2

,Title,Artist
0,adan y eva,Paulo Londra
16,it wont kill ya,The Chainsmokers - Louane
39,hymn,Kesha
58,arrows,Foo Fighters
80,talking to myself,Linkin Park
...,...,...
83794,lush life,Zara Larsson
83826,the business,Tiësto
83839,scooby doo pa pa,Dj Kass
83845,positions,Ariana Grande


In [374]:
enc.fit(X2)

OneHotEncoder()

In [375]:
enc.categories_

[array(['!', '#proudcatownerremix', '$$$ - with matt ox', ...,
        'è sempre bello', 'échame la culpa', '美女と野獣'], dtype=object),
 array(['*NSYNC',
        '13 Organisé - SCH - Kofs - Jul - Naps - Soso Maness - Elams - Solda - Houari',
        '187 Strassenbande - Bonez MC - Gzuz', ..., 'tha Supreme - Salmo',
        'twocolors',
        'Şanışer - Kamufle - Mert Şenel - Mirac - Aga B - Defkhan - Aspova - Yeis Sensura - Sehabe - Deniz Tekin - Ozbi - Sokrat St - Tahribad-ı İsyan - Beta - Server Uraz - Hayki - Ados - Fuat'],
       dtype=object)]

In [376]:
X2enc = enc.transform(X2).toarray()

In [377]:
X2df = pd.DataFrame(X2enc, columns=enc.get_feature_names_out(X2.columns))

In [378]:
X2df

,Title_!,Title_#proudcatownerremix,Title_$$$ - with matt ox,Title_+linda,Title_...ready for it?,Title_0.00,Title_00:00,"Title_1, 2, 3",Title_1-800-273-8255,Title_1.5,...,Artist_j-hope,Artist_j-hope - Becky G,Artist_jxdn,Artist_salem ilese,Artist_tha Supreme,Artist_tha Supreme - Mara Sattei,Artist_tha Supreme - Marracash,Artist_tha Supreme - Salmo,Artist_twocolors,Artist_Şanışer - Kamufle - Mert Şenel - Mirac - Aga B - Defkhan - Aspova - Yeis Sensura - Sehabe - Deniz Tekin - Ozbi - Sokrat St - Tahribad-ı İsyan - Beta - Server Uraz - Hayki - Ados - Fuat
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5419,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5420,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5421,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [379]:
sub_df = X2df.filter(regex='Artist_')

In [380]:
sub_df

,Artist_*NSYNC,Artist_13 Organisé - SCH - Kofs - Jul - Naps - Soso Maness - Elams - Solda - Houari,Artist_187 Strassenbande - Bonez MC - Gzuz,Artist_187 Strassenbande - Bonez MC - RAF Camora - Gzuz - Maxwell,Artist_187 Strassenbande - Gzuz - Bonez MC - LX,Artist_2 Chainz - Ariana Grande,Artist_2 Chainz - Drake - Quavo,Artist_2 Chainz - Gucci Mane - Quavo,Artist_2 Chainz - Kendrick Lamar,Artist_2 Chainz - Travis Scott,...,Artist_j-hope,Artist_j-hope - Becky G,Artist_jxdn,Artist_salem ilese,Artist_tha Supreme,Artist_tha Supreme - Mara Sattei,Artist_tha Supreme - Marracash,Artist_tha Supreme - Salmo,Artist_twocolors,Artist_Şanışer - Kamufle - Mert Şenel - Mirac - Aga B - Defkhan - Aspova - Yeis Sensura - Sehabe - Deniz Tekin - Ozbi - Sokrat St - Tahribad-ı İsyan - Beta - Server Uraz - Hayki - Ados - Fuat
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5419,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5420,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5421,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [381]:
X2df['Artist_Drake'].sum()

66.0

In [60]:
X1[['Explicit', 'danceability', 'energy', 'loudness', 'liveliness', 'valence', 'tempo', 'duration_ms', 'acoustics']].reset_index()

,index,Explicit,danceability,energy,loudness,liveliness,valence,tempo,duration_ms,acoustics
0,0,False,0.767,0.709,-4.47,0.0676,0.72,171.993,258639,0.323
1,16,False,0.572,0.53,-8.521,0.127,0.12,170.138,217613,0.0647
2,39,True,0.488,0.538,-4.974,0.305,0.38,67.037,205600,0.147
3,58,False,0.515,0.917,-7.312,0.145,0.409,121.988,266187,0.000209
4,80,False,0.593,0.712,-6.325,0.128,0.473,124.013,231307,0.00184
...,...,...,...,...,...,...,...,...,...,...
5418,83794,False,0.694,0.712,-3.923,0.211,0.799,98.022,201122,0.133
5419,83826,False,0.798,0.62,-7.079,0.112,0.235,120.031,164000,0.414
5420,83839,False,0.91,0.754,-2.573,0.122,0.63,120.939,145972,0.0396
5421,83845,True,0.737,0.802,-4.771,0.0931,0.682,144.015,172325,0.468


In [129]:
X3 = pd.concat([song_data[['Explicit', 'danceability', 'energy', 'loudness', 'speechiness', 'instrumentalness', 'liveliness', 'valence', 'tempo', 'duration_ms', 'acoustics', 'Popu_max']].reset_index(drop=True), X2df.reset_index(drop=True)], axis=1)

In [132]:
pd.reset_option('max_columns')
X3

,Explicit,danceability,energy,loudness,speechiness,instrumentalness,liveliness,valence,tempo,duration_ms,...,Artist_j-hope,Artist_j-hope - Becky G,Artist_jxdn,Artist_salem ilese,Artist_tha Supreme,Artist_tha Supreme - Mara Sattei,Artist_tha Supreme - Marracash,Artist_tha Supreme - Salmo,Artist_twocolors,Artist_Şanışer - Kamufle - Mert Şenel - Mirac - Aga B - Defkhan - Aspova - Yeis Sensura - Sehabe - Deniz Tekin - Ozbi - Sokrat St - Tahribad-ı İsyan - Beta - Server Uraz - Hayki - Ados - Fuat
0,False,0.767,0.709,-4.47,0.336,0.0,0.0676,0.72,171.993,258639,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,False,0.572,0.53,-8.521,0.0654,0.000169,0.127,0.12,170.138,217613,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,True,0.488,0.538,-4.974,0.076,0.0,0.305,0.38,67.037,205600,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,False,0.515,0.917,-7.312,0.0417,0.000476,0.145,0.409,121.988,266187,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,False,0.593,0.712,-6.325,0.0286,0.0,0.128,0.473,124.013,231307,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5418,False,0.694,0.712,-3.923,0.046,0.0,0.211,0.799,98.022,201122,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5419,False,0.798,0.62,-7.079,0.232,0.0192,0.112,0.235,120.031,164000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5420,False,0.91,0.754,-2.573,0.239,0.00154,0.122,0.63,120.939,145972,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5421,True,0.737,0.802,-4.771,0.0878,0.0,0.0931,0.682,144.015,172325,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [133]:
y = song_data[['Genre_new']].reset_index(drop=True)
y

,Genre_new
0,hip hop
1,pop
2,pop
3,metal
4,metal
...,...
5418,pop
5419,house
5420,dance/electronic
5421,pop


In [134]:
X_train, X_test, y_train, y_test = train_test_split(X3, y, test_size=0.2)

In [135]:
clf = DecisionTreeClassifier(criterion='entropy')
clf.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy')

In [136]:
print("Training Error: " + str(1.0-clf.score(X_train, y_train)))
print("Test Error: " + str(1.0-clf.score(X_test, y_test)))

Training Error: 0.0
Test Error: 0.5576036866359446


In [138]:
clf = LinearSVC(max_iter = 10000, C=1)
#clf = svm.SVC(C=0.1, kernel='linear')
#clf = svm.SVC(C=1, kernel="poly", degree=3)
clf.fit(X_train, y_train.values.ravel())
print("Training Error: " + str(1.0-clf.score(X_train, y_train)))
print("Test Error: " + str(1.0-clf.score(X_test, y_test)))

/Users/jeetajmani/opt/miniconda3/envs/seng/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Training Error: 0.9866297833102813
Test Error: 0.9861751152073732


In [121]:
X4 = song_data[['Explicit', 'danceability', 'energy', 'loudness', 'speechiness', 'instrumentalness', 'liveliness', 'valence', 'tempo', 'duration_ms', 'acoustics', 'Popu_max']].reset_index(drop=True)
X4

,Explicit,danceability,energy,loudness,speechiness,instrumentalness,liveliness,valence,tempo,duration_ms,acoustics,Popu_max
0,False,0.767,0.709,-4.47,0.336,0.0,0.0676,0.72,171.993,258639,0.323,10
1,False,0.572,0.53,-8.521,0.0654,0.000169,0.127,0.12,170.138,217613,0.0647,72
2,True,0.488,0.538,-4.974,0.076,0.0,0.305,0.38,67.037,205600,0.147,131
3,False,0.515,0.917,-7.312,0.0417,0.000476,0.145,0.409,121.988,266187,0.000209,103
4,False,0.593,0.712,-6.325,0.0286,0.0,0.128,0.473,124.013,231307,0.00184,86
...,...,...,...,...,...,...,...,...,...,...,...,...
5418,False,0.694,0.712,-3.923,0.046,0.0,0.211,0.799,98.022,201122,0.133,104
5419,False,0.798,0.62,-7.079,0.232,0.0192,0.112,0.235,120.031,164000,0.414,160
5420,False,0.91,0.754,-2.573,0.239,0.00154,0.122,0.63,120.939,145972,0.0396,88
5421,True,0.737,0.802,-4.771,0.0878,0.0,0.0931,0.682,144.015,172325,0.468,1


In [118]:
pd.set_option('display.max_columns', None)
song_data.head()

,Country,Uri,Popularity,Title,Artist,Album/Single,Genre,Artist_followers,Explicit,Album,Release_date,Track_number,Tracks_in_album,danceability,energy,key,loudness,mode,speechiness,acoustics,instrumentalness,liveliness,valence,tempo,duration_ms,time_signature,Genre_new,Days_since_release,Released_after_2017,Explicit_false,Explicit_true,album,compilation,single,bolero,boy band,country,dance/electronic,else,funk,hip hop,house,indie,jazz,k-pop,latin,metal,opm,pop,r&b/soul,rap,reggae,reggaeton,rock,trap,syuzhet_norm,bing_norm,afinn_norm,nrc_norm,syuzhet,bing,afinn,nrc,anger,anticipation,disgust,fear,joy,sadness,surprise,trust,negative,positive,n_words,anger_norm,anticipation_norm,disgust_norm,fear_norm,joy_norm,sadness_norm,surprise_norm,trust_norm,negative_norm,positive_norm,anger_norm2,anticipation_norm2,disgust_norm2,fear_norm2,joy_norm2,sadness_norm2,surprise_norm2,trust_norm2,negative_norm2,positive_norm2,negative_bog_jr,positive_bog_jr,Bayes,Negative_Bayes,Neutral_Bayes,Positive_Bayes,LDA_Topic,Celebrate,Desire,Explore,Fun,Hope,Love,Nostalgia,Thug,bing_norm_negative,bing_norm_neutral,bing_norm_positive,Argentina,Australia,Austria,Belgium,Brazil,Canada,Chile,Colombia,Costa Rica,Denmark,Ecuador,Finland,France,Germany,Global,Indonesia,Ireland,Italy,Malaysia,Mexico,Netherlands,New Zealand,Norway,Peru,Philippines,Poland,Portugal,Singapore,Spain,Sweden,Switzerland,Taiwan,Turkey,UK,USA,Popu_max,Top10_dummy,Top50_dummy,Cluster
0,Global,https://open.spotify.com/track/6FyRXC8tJUh863J...,31833.95,adan y eva,Paulo Londra,single,argentine hip hop,11427104.0,False,Adan y Eva,2018-11-05,1,1,0.767,0.709,1,-4.47,1,0.336,0.323,0.0,0.0676,0.72,171.993,258639,4,hip hop,740.0,1.0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,1.0,1.0,global
16,Global,https://open.spotify.com/track/5xhJmd0I15jFcEd...,418.00,it wont kill ya,The Chainsmokers - Louane,album,dance pop,16931568.0,False,Memories...Do Not Open,2017-04-07,7,12,0.572,0.53,6,-8.521,0,0.0654,0.0647,0.000169,0.127,0.12,170.138,217613,4,pop,1314.0,1.0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1.0,-1.0,-1.0,-1.0,-2.0,-1.0,-7.0,-3.0,0.0,1.0,0.0,1.0,1.0,2.0,0.0,1.0,4.0,1.0,274.0,0.000000,0.003650,0.0,0.00365,0.00365,0.007299,0.000000,0.003650,0.014599,0.003650,0.000000,0.166667,0.0,0.166667,0.166667,0.333333,0.000000,0.166667,0.8,0.2,0.0,0.0,-1.0,1.0,0.0,0.0,Desire,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,72,0.0,0.0,global
39,Global,https://open.spotify.com/track/0wJJhRgdHhT93Z3...,56.00,hymn,Kesha,single,dance pop,5910376.0,True,Hymn,2017-08-03,1,1,0.488,0.538,6,-4.974,1,0.076,0.147,0.0,0.305,0.38,67.037,205600,4,pop,1199.0,1.0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,131,0.0,0.0,global
58,Global,https://open.spotify.com/track/6JSryEdVJTZq6YB...,78.40,arrows,Foo Fighters,album,alternative metal,8092392.0,False,Concrete and Gold,2017-09-15,7,11,0.515,0.917,1,-7.312,1,0.0417,0.000209,0.000476,0.145,0.409,121.988,266187,4,metal,1156.0,1.0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,103,0.0,0.0,global
80,Global,https://open.spotify.com/t

In [120]:
# number 1 songs
song_data[song_data['Popu_max'] == 1]

,Country,Uri,Popularity,Title,Artist,Album/Single,Genre,Artist_followers,Explicit,Album,Release_date,Track_number,Tracks_in_album,danceability,energy,key,loudness,mode,speechiness,acoustics,instrumentalness,liveliness,valence,tempo,duration_ms,time_signature,Genre_new,Days_since_release,Released_after_2017,Explicit_false,Explicit_true,album,compilation,single,bolero,boy band,country,dance/electronic,else,funk,hip hop,house,indie,jazz,k-pop,latin,metal,opm,pop,r&b/soul,rap,reggae,reggaeton,rock,trap,syuzhet_norm,bing_norm,afinn_norm,nrc_norm,syuzhet,bing,afinn,nrc,anger,anticipation,disgust,fear,joy,sadness,surprise,trust,negative,positive,n_words,anger_norm,anticipation_norm,disgust_norm,fear_norm,joy_norm,sadness_norm,surprise_norm,trust_norm,negative_norm,positive_norm,anger_norm2,anticipation_norm2,disgust_norm2,fear_norm2,joy_norm2,sadness_norm2,surprise_norm2,trust_norm2,negative_norm2,positive_norm2,negative_bog_jr,positive_bog_jr,Bayes,Negative_Bayes,Neutral_Bayes,Positive_Bayes,LDA_Topic,Celebrate,Desire,Explore,Fun,Hope,Love,Nostalgia,Thug,bing_norm_negative,bing_norm_neutral,bing_norm_positive,Argentina,Australia,Austria,Belgium,Brazil,Canada,Chile,Colombia,Costa Rica,Denmark,Ecuador,Finland,France,Germany,Global,Indonesia,Ireland,Italy,Malaysia,Mexico,Netherlands,New Zealand,Norway,Peru,Philippines,Poland,Portugal,Singapore,Spain,Sweden,Switzerland,Taiwan,Turkey,UK,USA,Popu_max,Top10_dummy,Top50_dummy,Cluster
1720,Global,https://open.spotify.com/track/3swc6WTsr7rl9Dq...,50735.25,psycho,Post Malone - Ty Dolla $ign,album,dfw rap,29173640.0,True,beerbongs & bentleys,2018-04-27,8,18,0.75,0.56,8,-8.094,1,0.105,0.546,0.0,0.111,0.459,140.06,221440,4,rap,932.0,1.0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1.0,-1.0,-1.0,-1.0,-1.85,-1.0,-3.0,-3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,293.0,0.000000,0.000000,0.000000,0.003413,0.000000,0.000000,0.000000,0.000000,0.010239,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.0,6.0,-1.0,1.0,0.0,0.0,Hope,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.0,1.0,global
2953,Global,https://open.spotify.com/track/4HBZA5flZLE435Q...,30763.80,stuck with u,Ariana Grande - Justin Bieber,single,dance pop,52571724.0,False,Stuck with U,2020-05-08,1,1,0.597,0.45,8,-6.658,1,0.0418,0.223,0.0,0.382,0.537,178.765,228482,3,pop,190.0,1.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1.0,-1.0,-1.0,-1.0,1.20,-2.0,-2.0,-1.0,3.0,2.0,1.0,3.0,2.0,2.0,2.0,1.0,4.0,3.0,332.0,0.009036,0.006024,0.003012,0.009036,0.006024,0.006024,0.006024,0.003012,0.012048,0.009036,0.187500,0.125000,0.062500,0.187500,0.125000,0.125000,0.125000,0.062500,0.571429,0.428571,0.0,0.0,-1.0,1.0,0.0,0.0,Desire,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.0,1.0,global
3628,Global,https://open.spotify.com/track/4R2kfaDFhslZEMJ...,11167.85,cardigan,Taylor Swift,album,dance pop,33485486.0,False,folklore,2020-07-24,2,16,0.613,0.581,0,-8.588,0,0.0424,0.537,0.000345,0.25,0.551,130.033,239560,4,pop,113.0,1.0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1.0,-1.0,1.0,0.0,1.80,-3.0,2.0,0.0,4.0,5.0,4.0,7.0,6.0,5.0,5.0,3.0,7.0,7.0,287.0,0.013937,0.017422,0.013937,0.024390,0.020906,0.017422,0.017422,0.010453,0.024390,0.024390,0.102564,0.128205,0.102564,0.179487,0.153846,0.128205,0.128205,0.076923,0.500000,0.500000,0.0,11.0,-1.0,1.0,0.0,0.0,Love,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.0,1.0,global
6203,Global,https://open.spotify.com/track/116H0KvKr2Zl4RP...,58972.05,mia,Bad Bunny - Drake,single,latin,26265604.0,False,MIA (feat. Drake),2018-10-11,1,1,0.817,0.539,6,-6.349,0,0.0621,0.0141,0.000496,0.099,0.158,97.062,210368,4,latin,765.0,1.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1.0,-1.0,-1.0,-1.0,-2.55,-2.0,-2.0,-3.0,4.0,0.0,2.0,4.0,1.0,2.0,1.0,1.0,6.0,3.0,228.0,0.0175

In [235]:
xcols = ['Explicit', 'danceability', 'energy', 'loudness', 'speechiness', 'instrumentalness', 'liveliness', 'valence', 'tempo', 'duration_ms', 'acoustics', 'Popu_max']
normalized_X = X4.values
std_scalor = preprocessing.StandardScaler()
X_scaled = std_scalor.fit_transform(normalized_X)
normalized_X = pd.DataFrame(X_scaled, columns=xcols)

In [236]:
XT = pd.concat([normalized_X.reset_index(drop=True), X2df.reset_index(drop=True)], axis=1)

In [239]:
XT

,Explicit,danceability,energy,loudness,speechiness,instrumentalness,liveliness,valence,tempo,duration_ms,...,Artist_j-hope,Artist_j-hope - Becky G,Artist_jxdn,Artist_salem ilese,Artist_tha Supreme,Artist_tha Supreme - Mara Sattei,Artist_tha Supreme - Marracash,Artist_tha Supreme - Salmo,Artist_twocolors,Artist_Şanışer - Kamufle - Mert Şenel - Mirac - Aga B - Defkhan - Aspova - Yeis Sensura - Sehabe - Deniz Tekin - Ozbi - Sokrat St - Tahribad-ı İsyan - Beta - Server Uraz - Hayki - Ados - Fuat
0,-0.947355,0.577183,0.432321,0.750346,1.635203,-0.161006,-0.809493,1.072883,1.659786,1.022513,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.947355,-0.748378,-0.628602,-0.802740,-0.582711,-0.158792,-0.383307,-1.613182,1.597588,0.227903,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.055571,-1.319389,-0.581186,0.557121,-0.495831,-0.161006,0.893817,-0.449220,-1.859329,-0.004770,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.947355,-1.135849,1.665126,-0.339230,-0.776963,-0.154771,-0.254160,-0.319394,-0.016854,1.168706,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.947355,-0.605625,0.450102,0.039170,-0.884335,-0.161006,-0.376132,-0.032880,0.051043,0.493135,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5418,-0.947355,0.080947,0.450102,0.960057,-0.741719,-0.161006,0.219381,1.426549,-0.820420,-0.091502,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5419,-0.947355,0.787913,-0.095177,-0.249901,0.782790,0.090489,-0.490930,-1.098353,-0.082471,-0.810497,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5420,-0.947355,1.549261,0.699034,1.477625,0.840164,-0.140834,-0.419181,0.669974,-0.052026,-1.159672,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5421,1.055571,0.373251,0.983527,0.634948,-0.399115,-0.161006,-0.626535,0.902766,0.721699,-0.649255,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [332]:
y['Genre_new'].unique()

array(['hip hop', 'pop', 'metal', 'indie', 'rap', 'funk', 'r&b/soul',
       'k-pop', 'rock', 'dance/electronic', 'latin', 'trap', 'country',
       'house', 'reggaeton', 'boy band', 'else', 'bolero'], dtype=object)

In [333]:
replace_dict = {
    'rap':'hip hop',
    'metal':'rock',
    'trap':'hip hop',
    'house': 'dance/electronic'
}

ybetter = y['Genre_new'].replace(replace_dict)

In [338]:
YDF = pd.DataFrame(ybetter)

In [340]:
YDF['Genre_new'].unique()

array(['hip hop', 'pop', 'rock', 'indie', 'funk', 'r&b/soul', 'k-pop',
       'dance/electronic', 'latin', 'country', 'reggaeton', 'boy band',
       'else', 'bolero'], dtype=object)

In [366]:
X_train, X_test, y_train, y_test = train_test_split(XT, YDF, test_size=0.2)
clf = DecisionTreeClassifier(criterion='entropy')
clf.fit(X_train, y_train)
print("Training Error: " + str(1.0-clf.score(X_train, y_train)))
print("Test Error: " + str(1.0-clf.score(X_test, y_test)))

Training Error: 0.0
Test Error: 0.43870967741935485


In [355]:
YDF.head(35)

,Genre_new
0,hip hop
1,pop
2,pop
3,rock
4,rock
5,hip hop
6,hip hop
7,pop
8,indie
9,pop


In [261]:
y.values[:10]

array([['hip hop'],
       ['pop'],
       ['pop'],
       ['metal'],
       ['metal'],
       ['hip hop'],
       ['hip hop'],
       ['pop'],
       ['indie'],
       ['pop']], dtype=object)

In [344]:
predictions = clf.predict(XT)

In [345]:
wrongans = (np.where(np.concatenate(YDF.values, axis=0) != predictions))[0]

In [347]:
len(wrongans)

455

In [348]:
wrongans

array([   8,   11,   16,   22,   31,   32,   40,   54,   67,   79,   88,
         92,   96,  111,  133,  142,  143,  157,  160,  164,  172,  176,
        183,  214,  240,  241,  256,  258,  272,  275,  289,  292,  295,
        319,  366,  383,  384,  395,  460,  469,  471,  478,  492,  505,
        510,  562,  565,  574,  576,  593,  610,  631,  649,  650,  653,
        706,  714,  724,  752,  754,  756,  757,  765,  776,  795,  827,
        841,  855,  862,  884,  893,  894,  911,  947,  959,  964,  966,
        972,  975,  978,  979,  997, 1012, 1020, 1021, 1028, 1033, 1055,
       1075, 1078, 1098, 1099, 1170, 1172, 1178, 1185, 1187, 1190, 1192,
       1197, 1198, 1199, 1202, 1207, 1208, 1215, 1218, 1227, 1269, 1295,
       1296, 1304, 1307, 1311, 1328, 1342, 1355, 1356, 1357, 1371, 1378,
       1382, 1399, 1403, 1431, 1434, 1448, 1459, 1473, 1491, 1495, 1503,
       1518, 1544, 1608, 1616, 1646, 1652, 1660, 1664, 1668, 1673, 1683,
       1693, 1701, 1716, 1753, 1779, 1780, 1799, 18

In [354]:
predictions[31]

'boy band'

In [370]:
X_train, X_test, y_train, y_test = train_test_split(XT, YDF, test_size=0.2)
clf = LinearSVC(max_iter = 10000, C=1)
#clf = svm.SVC(C=0.1, kernel='linear')
#clf = svm.SVC(C=1, kernel="poly", degree=3)
clf.fit(X_train, y_train.values.ravel())
print("Training Error: " + str(1.0-clf.score(X_train, y_train)))
print("Test Error: " + str(1.0-clf.score(X_test, y_test)))

Training Error: 0.0
Test Error: 0.16866359447004609


## Trying genre to genre_new experiments

In [149]:
G_DF = song_data[['Genre', 'Explicit']].reset_index(drop=True)
GN_DF = song_data['Genre_new'].reset_index(drop=True)

In [151]:
encG = preprocessing.OneHotEncoder()

encG.fit(G_DF)

G_DF_enc = encG.transform(G_DF).toarray()

G_DF2 = pd.DataFrame(G_DF_enc, columns=encG.get_feature_names_out(G_DF.columns))

In [156]:
G_DF2

,Genre_a cappella,Genre_acoustic pop,Genre_adult standards,Genre_afro dancehall,Genre_afrofuturism,Genre_afrofuturismo brasileiro,Genre_afroswing,Genre_alabama rap,Genre_albanian hip hop,Genre_album rock,...,Genre_trap,Genre_trap latino,Genre_tropical house,Genre_turkish hip hop,Genre_uk hip hop,Genre_vapor pop,Genre_venezuelan hip hop,Genre_viral rap,Explicit_False,Explicit_True
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5419,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5420,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5421,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [157]:
X_train, X_test, y_train, y_test = train_test_split(G_DF2, GN_DF, test_size=0.2)

clf = DecisionTreeClassifier(criterion='entropy')
clf.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy')

In [162]:
print("Training Error: " + str(1.0-clf.score(X_train, y_train)))
print("Test Error: " + str(1.0-clf.score(X_test, y_test)))

Training Error: 0.0
Test Error: 0.03594470046082954


In [ ]:
### Jeet's Tests
song_data1 = song_data[song_data.iloc[:, 0].str.contains("Global") == True]

#song_data1 = song_data1[song_data1['Genre_new'] != 'n-a']
# Binary Classificati
#song_data1 = song_data1[song_data1.iloc[:, 26].str.contains("hip hop|rap") == True]
le = preprocessing.LabelEncoder()
Xj = song_data1[['Title', 'Artist', 'Explicit', 'danceability', 'energy', 'loudness', 'liveliness', 'valence', 'tempo', 'duration_ms', 'acoustics']]
Xj = Xj.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')
yj = song_data1[['Genre_new']]
yj = yj.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')

X_trainj, X_testj, y_trainj, y_testj = train_test_split(Xj, yj, test_size=0.2)

clfj = DecisionTreeClassifier()
clfj.fit(X_trainj, y_trainj)
print("Training Error: " + str(1.0-clfj.score(X_trainj, y_trainj)))
print("Test Error: " + str(1.0-clfj.score(X_testj, y_testj)))

Training Error: 0.0
Test Error: 0.3419354838709677


In [ ]:
test_errors = []
train_errors = []

for i in range(0,100):
  song_data1 = song_data[song_data.iloc[:, 0].str.contains("Global") == True]

  #song_data1 = song_data1[song_data1['Genre_new'] != 'n-a']
  # Binary Classificati
  song_data1 = song_data1[song_data1.iloc[:, 26].str.contains("hip hop|metal") == True]
  le = preprocessing.LabelEncoder()
  Xj = song_data1[['Title', 'Artist', 'Explicit', 'danceability', 'energy', 'loudness', 'liveliness', 'valence', 'tempo', 'duration_ms', 'acoustics']]
  Xj = Xj.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')
  yj = song_data1[['Genre_new']]
  yj = yj.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')

  X_trainj, X_testj, y_trainj, y_testj = train_test_split(Xj, yj, test_size=0.2)

  clfj = DecisionTreeClassifier()
  clfj.fit(X_trainj, y_trainj)

  train_errors.append(1.0-clfj.score(X_trainj, y_trainj))
  test_errors.append(1.0-clfj.score(X_testj, y_testj))

print("Training Error Average: " + str(np.average(train_errors)))
print("Test Error Average: " + str(np.average(test_errors)))

# Training Error Average: 0.0
# Test Error Average: 0.038741721854304634

Training Error Average: 0.0
Test Error Average: 0.038741721854304634


In [ ]:
song_data1 = song_data[song_data.iloc[:, 0].str.contains("Global") == True]

song_data1 = song_data1[song_data1['Genre'] != 'n-a']
le = preprocessing.LabelEncoder()
Xj = song_data1[['Title', 'Artist', 'Explicit', 'danceability', 'energy', 'loudness', 'liveliness', 'valence', 'tempo']]
Xj = Xj.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')
yj = song_data1[['Genre']]
yj = yj.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')
X_trainj, X_testj, y_trainj, y_testj = train_test_split(Xj, yj, test_size=0.2)

clfj = DecisionTreeClassifier()
clfj.fit(X_trainj, y_trainj)
print("Training Error: " + str(1.0-clfj.score(X_trainj, y_trainj)))
print("Test Error: " + str(1.0-clfj.score(X_testj, y_testj)))

Training Error: 0.0
Test Error: 0.40737327188940087


In [ ]:
#Normalize data in X 

#Minmax scalor preprocessing
#normalized_X=(X-X.min())/(X.max()-X.min())

#Standard scalor preprocessing
xcols = ['Title', 'Artist', 'Explicit', 'danceability', 'energy', 'loudness', 'liveliness', 'valence', 'tempo', 'duration_ms', 'acoustics']
#xcols = ['Artist', 'Explicit', 'danceability', 'energy', 'loudness', 'liveliness', 'valence', 'tempo', 'duration_ms', 'acoustics']
normalized_X = X.values
std_scalor = preprocessing.StandardScaler()
X_scaled = std_scalor.fit_transform(normalized_X)
normalized_X = pd.DataFrame(X_scaled, columns=xcols)

X_train3, X_test3, y_train3, y_test3 = train_test_split(normalized_X, y, test_size=0.2)

In [ ]:
#Reducing dataset size - not used in cells below anymore
X_train2 = X_train.iloc[0:7000,:]
X_test2 = X_test.iloc[0:1200,:]
y_train2 = y_train.iloc[0:7000,:]
y_test2 = y_test.iloc[0:1200,:]

In [ ]:
#Logistic Refression


#clf = LogisticRegression(multi_class='ovr', max_iter=10000)
clf = LogisticRegression(C=1, solver="lbfgs", penalty="l2", max_iter = 100000000)
clf.fit(X_train,y_train.values.ravel())
#train_accuracy = cross_val_score(clf, X_train2, y_train2).mean()
#test_accuracy = clf.score(X_test2, y_test2)

print("Training Error: " + str(1.0-clf.score(X_train, y_train)))
print("Test Error: " + str(1.0-clf.score(X_test, y_test)))

Training Error: 0.5198248040571691
Test Error: 0.5004608294930876


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
#Support Vector Machine

clf = LinearSVC(max_iter = 10000, C=1)
#clf = svm.SVC(C=0.1, kernel='linear')
#clf = svm.SVC(C=1, kernel="poly", degree=3)
clf.fit(X_train3, y_train3.values.ravel())
print("Training Error: " + str(1.0-clf.score(X_train3, y_train3)))
print("Test Error: " + str(1.0-clf.score(X_test3, y_test3)))

Training Error: 0.5154449054863992
Test Error: 0.5235023041474655


In [ ]:
#Decision tree
clf = DecisionTreeClassifier()
clf.fit(X_train3, y_train3.values.ravel())
print("Training Error: " + str(1.0-clf.score(X_train3, y_train3)))
print("Test Error: " + str(1.0-clf.score(X_test3, y_test3)))

Training Error: 0.0
Test Error: 0.3354838709677419


In [ ]:
#Random Forest # not enough ram to run on full dataset- change params
clf = RandomForestClassifier()
clf.fit(X_train3, y_train3.values.ravel())
print("Training Error: " + str(1.0-clf.score(X_train3, y_train3)))
print("Test Error: " + str(1.0-clf.score(X_test3, y_test3)))

Training Error: 0.0
Test Error: 0.3640552995391705


In [ ]:
clf = KNeighborsClassifier()
clf.fit(X_train3, y_train3.values.ravel())
print("Training Error: " + str(1.0-clf.score(X_train3, y_train3)))
print("Test Error: " + str(1.0-clf.score(X_test3, y_test3)))

Training Error: 0.3886583679114799
Test Error: 0.5612903225806452
